In [1]:
import urllib.request 
import urllib.parse
import urllib.error
import ssl
from itertools import chain
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as BS
import re
import pandas as pd

In [36]:


Street=[] 
City=[]
Zip=[]
Beds=[]
Baths=[]
Sqft=[]
Type=[]
Year=[]
Price=[]

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url='https://www.zillow.com/concord-ca/sold/?searchQueryState={%22pagination%22:{},%22mapBounds%22:{%22west%22:-122.06625938415527,%22east%22:-121.88429832458496,%22south%22:37.889418367596576,%22north%22:38.00908059939078},%22regionSelection%22:[{%22regionId%22:51518,%22regionType%22:6}],%22mapZoom%22:12,%22isMapVisible%22:true,%22filterState%22:{%22isRecentlySold%22:{%22value%22:true},%22isForSaleByAgent%22:{%22value%22:false},%22isForSaleByOwner%22:{%22value%22:false},%22isNewConstruction%22:{%22value%22:false},%22isComingSoon%22:{%22value%22:false},%22isAuction%22:{%22value%22:false},%22isForSaleForeclosure%22:{%22value%22:false},%22isPreMarketForeclosure%22:{%22value%22:false},%22isPreMarketPreForeclosure%22:{%22value%22:false}},%22isListVisible%22:true}'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()##read url as plaint text into webpage
soup = BS(webpage,'html.parser')

df=pd.DataFrame(([i['href']]for i in 
                  soup.findAll('a', attrs={'class':'list-card-link list-card-img'})),columns=['Link'])

links=df.values.tolist()
a=list(chain(*links))


for i, u in enumerate(a):
    print(i)
    
    req1 = Request(u, headers={'User-Agent': 'Mozilla/5.0'})
    webpage1 = urlopen(req1).read()##read url as plaint text into webpage
    text=webpage1.decode(encoding='utf8',errors='ignore')
    
    #Stree
    h='<div class="zsg-h1 hdp-home-header-st-addr">(.+?)</div>'
    patten_h=re.compile(h)
    value_h=re.findall(patten_h,text)
    if value_h:
        Street.append(value_h[0])
    else:
        Street.append(None)
    
#     print(value_h)

    #Zip Code
    z='<div class="zsg-h2">(.+?)</div>'
    patten_z=re.compile(z)
    value_z=re.findall(patten_z,text)
    
    if value_z:
        #city

        value_city=value_z[0].split(',')[0]
        City.append(value_city)

        #zip
        value_zip=value_z[0].split(' ')[2]
        Zip.append(value_zip)
    else:
        City.append(None)
        Zip.append(None)
    
    #Beds,baths, sqtf
    b='<h3 class="edit-facts-light">(.+?)</h3>'
    patten_b=re.compile(b)
    value_b=re.findall(patten_b,text) 
    
    if value_b:
        
        #beds
        value_beds=value_b[0].split(' ')[2].split('>')[3]
        Beds.append(value_beds)
    
        #baths
        value_baths=value_b[0].split(' ')[5].split('>')[3]
        Baths.append(value_baths)

        #sqft
        value_sqft=value_b[0].split(' ')[8].split('>')[3]
        Sqft.append(value_sqft)
    else:
        Beds.append(None)
        Baths.append(None)
        Sqft.append(None)

    #fact-value
    e='<div class="fact-value">(.+?)</div>'
    patten_e=re.compile(e)
    value_e=re.findall(patten_e,text)
    if value_e:
        Type.append(value_e[0])
        Year.append(value_e[1])
    else:
        Type.append(None)
        Year.append(None)
    
#     print(value_e)

       
    #sold price:
    f='<div class="status">(.+?)</div>'
    patten_f=re.compile(f)
    value_f=re.findall(patten_f,text)
    if value_f:
        value_sold=value_f[0].split('>')[5].split('<')[0].split('$')[1]
        Price.append(value_sold)
    else:
        Price.append(None)

# print('testing')
# print(Street)
# print(City)
# print(Zip)
# print(Beds)
# print(Baths)
# print(Year)
# print(Sqft)
# print(Type)
# print(Price)

# len(Street)
# len(Price)


    
df1=pd.DataFrame({'Street':Street, 'City':City,'Zip code':Zip,'Bed':Beds, 'Bath':Baths, 'Year Built':Year,
                 'Sqft':Sqft,'House Type':Type,'Sold Price':Price})

df1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


,Street,City,Zip code,Bed,Bath,Year Built,Sqft,House Type,Sold Price
0,4155 Lillian Dr,Concord,94521,3,2,1965,"1,344",Single Family,"570,000"
1,1795 Klier Dr,Concord,94518,5,3,1965,"2,210",Single Family,"830,000"
2,1630 Emma Ct,Concord,94519,3,2,1949,"1,250",Single Family,"650,000"
3,4377 Rose Ln,Concord,94518,4,3,2014,"2,828",Single Family,"1,111,000"
4,None,None,None,None,None,None,None,None,None
5,None,None,None,None,None,None,None,None,None
6,3815 Northwood Dr UNIT H,Concord,94520,2,2,1974,"1,088",Condo,"338,000"
7,None,None,None,None,None,None,None,None,None
8,None,None,None,None,None,None,None,None,None
9,4499 Big Pine Ln,Concord,94521,4,3,1975,"2,307",Single Family,"750,000"


In [3]:
import urllib.request 
import urllib.parse
import urllib.error
import ssl
from itertools import chain
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as BS
import re
import pandas as pd

Street=[] 
City=[]
Zip=[]
Beds=[]
Baths=[]
Sqft=[]
Type=[]
Year=[]
Price=[]

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url='https://www.zillow.com/concord-ca/sold/5_p/?searchQueryState={%22pagination%22:{%22currentPage%22:5},%22mapBounds%22:{%22west%22:-122.17252005957032,%22east%22:-121.80859794042969,%22south%22:37.873646305099996,%22north%22:38.112827121594876},%22regionSelection%22:[{%22regionId%22:51518,%22regionType%22:6}],%22isMapVisible%22:true,%22mapZoom%22:11,%22filterState%22:{%22isRecentlySold%22:{%22value%22:true},%22isForSaleByAgent%22:{%22value%22:false},%22isForSaleByOwner%22:{%22value%22:false},%22isNewConstruction%22:{%22value%22:false},%22isComingSoon%22:{%22value%22:false},%22isAuction%22:{%22value%22:false},%22isForSaleForeclosure%22:{%22value%22:false},%22isPreMarketForeclosure%22:{%22value%22:false},%22isPreMarketPreForeclosure%22:{%22value%22:false}},%22isListVisible%22:true}'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()##read url as plaint text into webpage
soup = BS(webpage,'html.parser')

df=pd.DataFrame(([i['href']]for i in 
                  soup.findAll('a', attrs={'class':'list-card-link list-card-img'})),columns=['Link'])

links=df.values.tolist()
a=list(chain(*links))

for u in a:
    req1 = Request(u, headers={'User-Agent': 'Mozilla/5.0'})
    webpage1 = urlopen(req1).read()##read url as plaint text into webpage
    text=webpage1.decode(encoding='utf8',errors='ignore')
    
    #Stree
    h='<div class="zsg-h1 hdp-home-header-st-addr">(.+?)</div>'
    patten_h=re.compile(h)
    value_h=re.findall(patten_h,text)
    Street.extend(value_h)


    #Zip Code
    z='<div class="zsg-h2">(.+?)</div>'
    patten_z=re.compile(z)
    value_z=re.findall(patten_z,text)
    
    #city
    value_city=value_z[0].split(',')[0]
    City.append(value_city)
     
    #zip
    value_zip=value_z[0].split(' ')[2]
    Zip.append(value_zip)
    
    
    #Beds,baths, sqtf
    b='<h3 class="edit-facts-light">(.+?)</h3>'
    patten_b=re.compile(b)
    value_b=re.findall(patten_b,text) 
    
    #beds
    value_beds=value_b[0].split(' ')[2].split('>')[3]
    Beds.append(value_beds)

    
    #baths
    value_baths=value_b[0].split(' ')[5].split('>')[3]
    Baths.append(value_baths)

    
    #sqft
    value_sqft=value_b[0].split(' ')[8].split('>')[3]
    Sqft.append(value_sqft)

    #fact-value
    e='<div class="fact-value">(.+?)</div>'
    patten_e=re.compile(e)
    value_e=re.findall(patten_e,text)
    Type.append(value_e[0])
    Year.append(value_e[1])
    
    #sold price:
    f='<div class="status">(.+?)</div>'
    patten_f=re.compile(f)
    value_f=re.findall(patten_f,text)
    value_sold=value_f[0].split('>')[5].split('<')[0].split('$')[1]
    Price.append(value_sold)


    
df2=pd.DataFrame({'Street':Street, 'City':City,'Zip code':Zip,'Bed':Beds, 'Bath':Baths, 'Year Built':Year,
                'Sqft':Sqft,'House Type':Type,'Sold Price':Price})

df2

IndexError: list index out of range

In [6]:
#drop dummy data for dataframe df2:
df2_d=df2.drop([1,9,27,28,39],axis=0)
df2_d

,Street,City,Zip code,Bed,Bath,Year Built,Sqft,House Type,Sold Price
0,1086 Mohr Ln APT B,Concord,94518,2,2,1972,877,Condo,"330,000"
2,164 Norman Ave,Concord,94520,3,2,1918,"1,113",Single Family,"485,000"
3,1001 Oak Grove Rd,Concord,94518,4,2,1960,"1,677",Single Family,"700,000"
4,3624 Chestnut Ave,Concord,94519,4,2,1941,"1,643",Single Family,"667,000"
5,1511 Parkwood Pl,Concord,94521,2,2,1978,"1,088",Condo,"339,900"
6,1519 Terry Lynn Ln,Concord,94521,4,3,1994,"1,977",Single Family,"730,000"
7,2751 Broadmoor Ave,Concord,94520,3,1,1952,"1,020",Single Family,"420,000"
8,5020 Valley Crest Dr APT 101,Concord,94521,2,2,1984,"1,080",Apartment,"355,000"
10,1069 Deer Oak Pl,Concord,94521,3,3,1991,"1,636",Condo,"572,000"
11,1042 Kaski Ln,Concord,94518,3,2,1954,"1,624",Single Family,"540,000"


In [8]:
import urllib.request 
import urllib.parse
import urllib.error
import ssl
from itertools import chain
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as BS
import re
import pandas as pd

Street=[] 
City=[]
Zip=[]
Beds=[]
Baths=[]
Sqft=[]
Type=[]
Year=[]
Price=[]

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url='https://www.zillow.com/concord-ca/sold/7_p/?searchQueryState={%22pagination%22:{%22currentPage%22:7},%22mapBounds%22:{%22west%22:-122.35448111914063,%22east%22:-121.62663688085938,%22south%22:37.780360754799815,%22north%22:38.205691086665915},%22regionSelection%22:[{%22regionId%22:51518,%22regionType%22:6}],%22isMapVisible%22:true,%22filterState%22:{%22isRecentlySold%22:{%22value%22:true},%22isForSaleByAgent%22:{%22value%22:false},%22isForSaleByOwner%22:{%22value%22:false},%22isNewConstruction%22:{%22value%22:false},%22isComingSoon%22:{%22value%22:false},%22isAuction%22:{%22value%22:false},%22isForSaleForeclosure%22:{%22value%22:false},%22isPreMarketForeclosure%22:{%22value%22:false},%22isPreMarketPreForeclosure%22:{%22value%22:false}},%22isListVisible%22:true}'
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read().decode()##read url as plaint text into webpage
soup = BS(webpage,'html.parser')

df=pd.DataFrame(([i['href']]for i in 
                  soup.findAll('a', attrs={'class':'list-card-link list-card-img'})),columns=['Link'])

links=df.values.tolist()
a=list(chain(*links))

for u in a:
    req1 = Request(u, headers={'User-Agent': 'Mozilla/5.0'})
    webpage1 = urlopen(req1).read()##read url as plaint text into webpage
    text=webpage1.decode(encoding='utf8',errors='ignore')
    
    #Stree
    h='<div class="zsg-h1 hdp-home-header-st-addr">(.+?)</div>'
    patten_h=re.compile(h)
    value_h=re.findall(patten_h,text)
    Street.extend(value_h)
   
    #Zip Code
    z='<div class="zsg-h2">(.+?)</div>'
    patten_z=re.compile(z)
    value_z=re.findall(patten_z,text)
   
    #city
    value_city=value_z[0].split(',')[0]
    City.append(value_city)
    
    #zip
    value_zip=value_z[0].split(' ')[2]
    Zip.append(value_zip)
    
    
    
    #Beds,baths, sqtf
    b='<h3 class="edit-facts-light">(.+?)</h3>'
    patten_b=re.compile(b)
    value_b=re.findall(patten_b,text) 
    
    #beds
    value_beds=value_b[0].split(' ')[2].split('>')[3]
    Beds.append(value_beds)

    
    #baths
    value_baths=value_b[0].split(' ')[5].split('>')[3]
    Baths.append(value_baths)

    
    #sqft
    value_sqft=value_b[0].split(' ')[8].split('>')[3]
    Sqft.append(value_sqft)

    #fact-value
    e='<div class="fact-value">(.+?)</div>'
    patten_e=re.compile(e)
    value_e=re.findall(patten_e,text)
    Type.append(value_e[0])
    Year.append(value_e[1])

    #sold price:
    f='<div class="status">(.+?)</div>'
    patten_f=re.compile(f)
    value_f=re.findall(patten_f,text)
    value_sold=value_f[0].split('>')[5].split('<')[0].split('$')[1]
    Price.append(value_sold)


df3=pd.DataFrame({'Street':Street, 'City':City,'Zip code':Zip,'Bed':Beds, 'Bath':Baths, 'Year Built':Year,
                'Sqft':Sqft,'House Type':Type,'Sold Price':Price})

df3

,Street,City,Zip code,Bed,Bath,Year Built,Sqft,House Type,Sold Price
0,1705 Trailside Cir,Concord,94518,3,3,2008,"1,332",Apartment,"565,000"
1,5516 Pamplona Ct,Concord,94521,3,2.5,1975,"2,041",Single Family,"585,500"
2,4982 Murchio Dr,Concord,94521,3,2,1983,"1,659",Single Family,"710,000"
3,1288 Pine Creek Way APT H,Concord,94520,2,2,1970,"1,089",Townhouse,"352,000"
4,4857 Cherokee Dr,Concord,94521,5,3,1961,"2,086",Single Family,"163,000"
5,4939 Laurel Dr,Concord,94521,5,5,1994,"3,892",Single Family,"1,132,500"
6,4628 Melody Dr APT F,Concord,94521,2,2,1988,899,Condo,"370,000"
7,968 Heathergreen Ct,Concord,94521,4,3,1997,"2,820",Single Family,"995,000"
8,2108 La Orinda Pl,Concord,94518,4,2.5,1975,"2,010",Single Family,"708,000"
9,4494 Stone Canyon Ct,Concord,94521,4,3,1976,"2,026",Single Family,"834,500"


In [9]:
#drop dummy data for dataframe df2:
df3_d=df3.drop([17,35],axis=0)
df3_d

,Street,City,Zip code,Bed,Bath,Year Built,Sqft,House Type,Sold Price
0,1705 Trailside Cir,Concord,94518,3,3,2008,"1,332",Apartment,"565,000"
1,5516 Pamplona Ct,Concord,94521,3,2.5,1975,"2,041",Single Family,"585,500"
2,4982 Murchio Dr,Concord,94521,3,2,1983,"1,659",Single Family,"710,000"
3,1288 Pine Creek Way APT H,Concord,94520,2,2,1970,"1,089",Townhouse,"352,000"
4,4857 Cherokee Dr,Concord,94521,5,3,1961,"2,086",Single Family,"163,000"
5,4939 Laurel Dr,Concord,94521,5,5,1994,"3,892",Single Family,"1,132,500"
6,4628 Melody Dr APT F,Concord,94521,2,2,1988,899,Condo,"370,000"
7,968 Heathergreen Ct,Concord,94521,4,3,1997,"2,820",Single Family,"995,000"
8,2108 La Orinda Pl,Concord,94518,4,2.5,1975,"2,010",Single Family,"708,000"
9,4494 Stone Canyon Ct,Concord,94521,4,3,1976,"2,026",Single Family,"834,500"


In [10]:
#combine all 3 dataframe together:
import pandas as pd
df_combine=pd.concat([df1,df2_d,df3_d],axis=0,ignore_index=True)
df_combine

,Street,City,Zip code,Bed,Bath,Year Built,Sqft,House Type,Sold Price
0,5381 Aspenwood Ct,Concord,94521,4,3,1998,"3,280",Single Family,"1,040,000"
1,5050 Valley Crest Dr APT 67,Concord,94521,2,2,1984,"1,100",Condo,"360,000"
2,1631 Yeoman Dr,Concord,94521,3,2,1963,"1,571",Single Family,"630,000"
3,4394 N Sugar Maple Ct,Concord,94521,4,2,1976,"1,917",Single Family,"864,500"
4,1801 Mayette Ave,Concord,94520,3,2,1949,"1,713",Single Family,"640,000"
...,...,...,...,...,...,...,...,...,...
108,1061 Mohr Ln APT C,Concord,94518,2,2,1973,880,Condo,"310,000"
109,2045 Highland Dr,Concord,94520,3,1,1951,998,Single Family,"468,000"
110,1071 White Oak Dr,Concord,94521,4,2,1960,"1,103",Single Family,"575,000"
111,1607 Lynwood Dr,Concord,94521,5,3,1963,"2,100",Single Family,"740,000"


In [11]:
#convert str into float:
df_combine['Sold Price'] = df_combine['Sold Price'].str.replace(',','').astype(float)
df_combine['Sqft']=df_combine['Sqft'].str.replace(',','').astype(float)
df_combine.dtypes

Street         object
City           object
Zip code       object
Bed            object
Bath           object
Year Built     object
Sqft          float64
House Type     object
Sold Price    float64
dtype: object

In [12]:
df_combine.to_csv('Concord.csv',index=False)